In [1]:
import pandas as pd
import nltk
import pymorphy2
import re
import numpy as np
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KoikovID\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df=pd.read_csv('test_data.csv')

In [4]:
def search_two_part_word(doc):
    doc=doc.lower()
    doc=doc.replace('до свидания', 'до_свидания')
    doc=doc.replace('до завтра', 'до_завтра')
    doc=doc.replace('добрый день', 'добрый_день')
    doc=doc.replace('добрый вечер', 'добрый_вечер')
    doc=doc.replace('доброе утро', 'доброе_утро')
    return doc
        

In [5]:
def search_name(doc):
    # probability score threshold
    prob_thresh = 0.4
    name=0
    morph = pymorphy2.MorphAnalyzer()
    for word in nltk.word_tokenize(doc.text):
        #print(word,"  aaaaaaaa")
        for p in morph.parse(word):
            #print(p,"   sssss")
            if 'Name' in p.tag and p.score >= prob_thresh:
                #print('{:<12}\t({:>12})\tscore:\t{:0.3}\t{}'.format(word, p.normal_form, p.score,doc['index']))
                name=word
                
    return name

In [6]:
def search_greeting_farewell(doc):
    dict_welcome=['здравствуйте','добрый_день']
    dict_goodbye=['до_свидания','пока']
    temp={}
    for token in doc.text.split():
        if token in dict_welcome:
            temp='welcome'
        elif token in dict_goodbye:
            temp='goodbye'
    return temp

In [7]:
df_for_lematize=df.text

In [8]:
df_for_lematize=df_for_lematize.apply(search_two_part_word)

In [9]:
serch_name=df_for_lematize.reset_index().apply(search_name,axis=1)

In [10]:
#serch_name

In [11]:
greet_farew=df_for_lematize.reset_index().apply(search_greeting_farewell,axis=1)

In [12]:
#serch_name

In [13]:
df['idx_name']=serch_name

In [14]:
df["welcome"] = ""
df["goodbye"] = ""

In [15]:
for i,el in enumerate(greet_farew):
    if el =='welcome':
        df['welcome'][i]=1
        df['goodbye'][i]=0
    elif el=="goodbye":
        df['welcome'][i]=0
        df['goodbye'][i]=1
    else:
        df['welcome'][i]=0
        df['goodbye'][i]=0


<ipython-input-15-a91333a2c5f2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['welcome'][i]=0
<ipython-input-15-a91333a2c5f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['goodbye'][i]=0
<ipython-input-15-a91333a2c5f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['welcome'][i]=1
<ipython-input-15-a91333a2c5f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

In [16]:
df_only_manager=df[df.role=='manager']

In [17]:
#df_only_manager

In [18]:
final={}
for i in df_only_manager.dlg_id.unique():
    temp={}
    #print( df_only_manager[['text','role']][df_only_manager.dlg_id==i][df_only_manager.welcome!=0])
    #print( df_only_manager[df_only_manager.dlg_id==i][df_only_manager.idx_name!=0])
    try:
        temp['welcome']=(df_only_manager.text[df_only_manager.dlg_id==i][df_only_manager.welcome!=0].values[0])  
    except:
        temp['welcome']='-'
    try:
         temp['goodbye']=(df_only_manager.text[df_only_manager.dlg_id==i][df_only_manager.goodbye!=0].values[0])
    except:
        temp['goodbye']='-'
    try:
        temp['name_phrase']=(df_only_manager.text[df_only_manager.dlg_id==i][df_only_manager.idx_name!=0].values[0])
    except:
        temp['name_phrase']='-'
    try:
        temp['name']=(df_only_manager.idx_name[df_only_manager.dlg_id==i][df_only_manager.idx_name!=0].values[0])
    except:
        temp['name']='-'
        
        
    final[i]=temp



In [19]:
final

{0: {'welcome': 'Алло здравствуйте',
  'goodbye': 'Всего хорошего до свидания',
  'name_phrase': 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
  'name': 'ангелина'},
 1: {'welcome': 'Алло здравствуйте',
  'goodbye': 'До свидания',
  'name_phrase': 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
  'name': 'ангелина'},
 2: {'welcome': 'Алло здравствуйте',
  'goodbye': '-',
  'name_phrase': 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
  'name': 'ангелина'},
 3: {'welcome': 'Алло дмитрий добрый день',
  'goodbye': '-',
  'name_phrase': 'Алло дмитрий добрый день',
  'name': 'дмитрий'},
 4: {'welcome': '-',
  'goodbye': 'Во вторник все ну с вами да тогда до вторника до свидания',
  'name_phrase': '-',
  'name': '-'},
 5: {'welcome': '-